# **Install KenLM**
Read [this](https://kheafield.com/code/kenlm/) for more info 

In [ ]:
!git clone https://github.com/kpu/kenlm.git
%cd kenlm
!mkdir -p build
%cd build
!cmake .. && make
%cd ../..

fatal: destination path 'kenlm' already exists and is not an empty directory.
/home/sadeghi/Notebooks/kenlm
/home/sadeghi/Notebooks/kenlm/build
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Configuring done
-- Generating done
-- Build files have been written to: /home/sadeghi/Notebooks/kenlm/build
Consolidate compiler generated dependencies of target kenlm_util
[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-conversion.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/io.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/line_input.cc.o
[ 10%] Building CXX object util/CMak

In [ ]:
import kenlm
from pyctcdecode import build_ctcdecoder
import torch

## **Make Language Model using KenLM**
In this section, we used cleaned, normalized, and processed Farsi text data to prepare a language model. It is better to use large-size text data and diverse in terms of structure and classification, which means that the data includes poetry, news, daily conversation, etc.

We have written a comprehensive data-cleaning program that I will add to the documentation.
KenLM also has some configurations such as `prune`.

using these options make your LM lighter and faster, improves your performance

In [ ]:
# raw_text.txt: a normalized text corpus (each line should contain a sentence)
!kenlm/build/bin/lmplz -o 5 \
    </home/sadeghi/totalck_UTF8_clean.txt \
    >/home/sadeghi/language_model_15G/5-gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/sadeghi/totalck_UTF8_clean.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
^C


In [ ]:
!kenlm/build/bin/build_binary \
    /home/sadeghi/language_model_15G/5-gram.arpa \
    /home/sadeghi/language_model_15G/5-gram.bin


This section wraps the model processor with LM that you can use later to transcript more accurately.

In [ ]:
from pyctcdecode import build_ctcdecoder
from transformers import AutoProcessor
from transformers import Wav2Vec2ProcessorWithLM

model_id = "/home/sadeghi/New_training_all_data/chekpoint-318000""

# LM must be in arpa format (for now)
lm_path = "/home/sadeghi/language_model_15G/5-gram.arpa"


processor = AutoProcessor.from_pretrained(model_id)
vocab_dict = processor.tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    lm_path,
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

processor_with_lm.save_pretrained("/home/sadeghi/language_model_15G/processor_with_lm_sadeghi")